In [0]:

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import csv
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

In [0]:


def Preprocessing(text):
  # text=text.decode('utf8')
    tokens=[word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    stops=stopwords.words('english')
    tokens=[t for t in tokens if t not in stops]
    tokens=[token for token in tokens if len(token)>=3]
    tokens=[word.lower() for word in tokens]
    lmtzr=WordNetLemmatizer()
    tokens=[lmtzr.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

def Classifier():
    classifier=Sequential()
    classifier.add(Dense(units=5907,input_shape=(5907,),activation='relu'))
    classifier.add(Dense(units=200,activation='relu'))
    classifier.add(Dense(units=2,activation='softmax'))
    classifier.compile(optimizer='adam',loss='categorical_crossentropy')
    return classifier

In [0]:
smsdata=open(list(uploaded.keys())[0])
sms_data=[]
sms_label=[]
csv_data=csv.reader(smsdata,delimiter='\t')
for line in csv_data:
    sms_label.append(line[0])
    sms_data.append(Preprocessing(line[1]))

In [0]:
import pickle

In [0]:
train_set_size=int(round(len(sms_data)*0.70))
x_train=np.array([''.join(el) for el in sms_data[0:train_set_size]])
y_train=np.array([el for el in sms_label[0:train_set_size]])
x_test=np.array([''.join(el) for el in sms_data[train_set_size+1:len(sms_data)]])
y_test=np.array([el for el in sms_label[train_set_size+1:len(sms_label)]])
le = preprocessing.LabelEncoder()
y_train=le.fit_transform(y_train)
ohe=preprocessing.OneHotEncoder()
y_train=ohe.fit_transform(np.reshape(y_train,(-1,1)))
pickle.dump(le,open('encoding','wb'))

In [0]:
tvect = TfidfVectorizer(min_df=2, ngram_range=(1, 2), stop_words='english', strip_accents='unicode', norm='l2')
X_train=tvect.fit_transform(x_train)
X_test=tvect.transform(x_test)
pickle.dump(tvect,open('vect','wb'))

In [0]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [0]:

Multiclass_Classifier=Classifier()
Multiclass_Classifier.fit(X_train,y_train,batch_size=500,epochs=100)
y_pred=Multiclass_Classifier.predict(X_test)

Epoch 1/100
3900/3900 [==============================] - 2s 449us/step - loss: 0.4267
Epoch 2/100
3900/3900 [==============================] - 1s 164us/step - loss: 0.1770
Epoch 3/100
3900/3900 [==============================] - 1s 168us/step - loss: 0.1158
Epoch 4/100
3900/3900 [==============================] - 1s 169us/step - loss: 0.1048
Epoch 5/100
3900/3900 [==============================] - 1s 166us/step - loss: 0.0977
Epoch 6/100
3900/3900 [==============================] - 1s 169us/step - loss: 0.0927
Epoch 7/100
3900/3900 [==============================] - 1s 169us/step - loss: 0.0859
Epoch 8/100
3900/3900 [==============================] - 1s 172us/step - loss: 0.0755
Epoch 9/100
3900/3900 [==============================] - 1s 169us/step - loss: 0.0575
Epoch 10/100
3900/3900 [==============================] - 1s 170us/step - loss: 0.0335
Epoch 11/100
3900/3900 [==============================] - 1s 169us/step - loss: 0.0157
Epoch 12/100
3900/3900 [============================

3000/3900 [======================>.......] - ETA: 0s - loss: 1.9110e-043900/3900 [==============================] - 1s 168us/step - loss: 0.0015
Epoch 92/100
3900/3900 [==============================] - 1s 166us/step - loss: 0.0014
Epoch 93/100
3900/3900 [==============================] - 1s 166us/step - loss: 0.0014
Epoch 94/100
3900/3900 [==============================] - 1s 167us/step - loss: 0.0015
Epoch 95/100
3900/3900 [==============================] - 1s 167us/step - loss: 0.0014
Epoch 96/100
3900/3900 [==============================] - 1s 167us/step - loss: 0.0015
Epoch 97/100
3900/3900 [==============================] - 1s 169us/step - loss: 0.0014
Epoch 98/100
3900/3900 [==============================] - 1s 168us/step - loss: 0.0014
Epoch 99/100
3900/3900 [==============================] - 1s 166us/step - loss: 0.0014
Epoch 100/100
3900/3900 [==============================] - 1s 171us/step - loss: 0.0014


In [0]:
Multiclass_Classifier.save('model.h5')

In [0]:
print(Multiclass_Classifier.predict(tvect.transform([Preprocessing("Bumper offer only for you.Hurry up,Dont miss the chance")])))

[[0.60491484 0.39508516]]


In [0]:
from keras.models import load_model
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


def Preprocessing(text):
  # text=text.decode('utf8')
    tokens=[word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    stops=stopwords.words('english')
    tokens=[t for t in tokens if t not in stops]
    tokens=[token for token in tokens if len(token)>=3]
    tokens=[word.lower() for word in tokens]
    lmtzr=WordNetLemmatizer()
    tokens=[lmtzr.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

classifier=load_model('model.h5')
tvect=pickle.load(open('vect','rb'))
le=pickle.load(open('encoding','rb'))
print(classifier.predict_classes(tvect.transform([Preprocessing("Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030")])))

[1]
